In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DataframeExercise").getOrCreate()



In [2]:
spark.sparkContext

<SparkContext master=local[*] appName=DataframeExercise>

In [3]:

from pyspark.sql import SQLContext


In [4]:
from pyspark.sql.functions import split
from pyspark.sql.functions import col

In [5]:
from pyspark.sql.functions import regexp_replace

Market Analysis in Banking Domain
Background and Objective:
Your client, a Portuguese banking institution, ran a marketing campaign to convince potential 
customers to invest in a bank term deposit scheme.
The marketing campaigns were based on phone calls. Often, the same customer was 
contacted more than once through phone, in order to assess if they would want to subscribe to 
the bank term deposit or not. You have to perform the marketing analysis of the data 
generated by this campaign.
Domain: Banking (Market Analysis)
Dataset Description
The data fields are as follows:
1. age numeric
2. job
type of job (categorical: 'admin.','bluecollar','entrepreneur','housemaid','management','retired','selfemployed','services','student','technician','unemployed','unknown')
3. marital marital status (categorical: 'divorced', 'married', 'single', 'unknown'; note: 
'divorced' means divorced or widowed)
4. education 
(categorical:basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.cour
se','university.degree','unknown')
5. default has credit in default? (categorical: 'no', 'yes', 'unknown')
6. housing: has housing loan? (categorical: 'no', 'yes', 'unknown')
7. loan has a personal loan? (categorical: 'no', 'yes', 'unknown')
# related to the last contact of the current campaign:
8. contact contact communication type (categorical: 'cellular', 'telephone')
9. month Month of last contact (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10. day_of_week last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11. duration
last contact duration, in seconds (numeric). Important note: this 
attribute highly affects the output target (example, if duration=0 
then y='no'). Yet, the duration is not known before a call is 
performed. Also, after the end of the call “y” is obviously known. 
Thus, this input should only be included for benchmark purposes 
and should be discarded if the intention is to have a realistic 
predictive model.
# other attributes:
12. campaign number of times a customer was contacted during the campaign 
(numeric, includes last contact)
13. pdays: 
number of days passed after the customer was last contacted from a 
previous campaign (numeric; 999 means customer was not 
previously contacted)
14. previous number of times the customer was contacted prior to (or before) this 
campaign (numeric)
15. poutcome outcome of the previous marketing campaign (categorical: 'failure', 
'nonexistent', 'success')
#Output variable (desired target):
16 y has the customer subscribed a term deposit? (binary: 'yes', 'no')
Analysis tasks to be done:
The data size is huge and the marketing team has asked you to perform the below analysis1. Load data and create a Spark data frame
2. Give marketing success rate (No. of people subscribed / total no. of entries)
 Give marketing failure rate
1. Give the maximum, mean, and minimum age of the average targeted customer
2. Check the quality of customers by checking average balance, median balance of 
customers
3. Check if age matters in marketing subscription for deposit
4. Check if marital status mattered for a subscription to deposit
5. Check if age and marital status together mattered for a subscription to deposit scheme
6. Do feature engineering for the bank and find the right age effect on the campaign.

## 1. Load data and create a Spark data frame


In [18]:
bank = spark.read.csv("Project1_dataset_bank.csv",inferSchema=True,header=True,sep=";")

In [11]:
bank = spark.read.format("csv").option("inferSchema", 'true').option("header", 'true').option('sep',",").load("Project1_dataset_bank.csv")

In [12]:
SQLContext(sparkContext=spark.sparkContext, sparkSession=spark)

In [13]:
#sqlContext = SQLContext(sc)

In [14]:
#bank.createOrReplaceTempView("bank_temp")

In [15]:
#x = SQLContext.sql("select * from bank_temp")

In [19]:
bank.show(2)

+------------------+-----------+-------------+-----------+-----------+-----------+--------+-----------+-------+---------+------------+------------+---------+------------+------------+-------+
|      "age;""job""|""marital""|""education""|""default""|""balance""|""housing""|""loan""|""contact""|""day""|""month""|""duration""|""campaign""|""pdays""|""previous""|""poutcome""| ""y"""|
+------------------+-----------+-------------+-----------+-----------+-----------+--------+-----------+-------+---------+------------+------------+---------+------------+------------+-------+
|"58;""management""|""married""| ""tertiary""|     ""no""|       2143|    ""yes""|  ""no""|""unknown""|      5|  ""may""|         261|           1|       -1|           0| ""unknown""|""no"""|
|"44;""technician""| ""single""|""secondary""|     ""no""|         29|    ""yes""|  ""no""|""unknown""|      5|  ""may""|         151|           1|       -1|           0| ""unknown""|""no"""|
+------------------+-----------+--------

In [20]:
bank.select(bank['""marital""']).show()

+------------+
| ""marital""|
+------------+
| ""married""|
|  ""single""|
| ""married""|
| ""married""|
|  ""single""|
| ""married""|
|  ""single""|
|""divorced""|
| ""married""|
|  ""single""|
|""divorced""|
|  ""single""|
| ""married""|
| ""married""|
| ""married""|
| ""married""|
|  ""single""|
| ""married""|
| ""married""|
| ""married""|
+------------+
only showing top 20 rows



In [21]:
new_col_names = ['age_job','marital','education','default','balance','housing','loan','contact','day','month','duration','campaign','pdays','previous','poutcome','y']
df = bank.toDF(*new_col_names)

In [22]:
df.show(2)

+------------------+-----------+-------------+-------+-------+-------+------+-----------+---+-------+--------+--------+-----+--------+-----------+-------+
|           age_job|    marital|    education|default|balance|housing|  loan|    contact|day|  month|duration|campaign|pdays|previous|   poutcome|      y|
+------------------+-----------+-------------+-------+-------+-------+------+-----------+---+-------+--------+--------+-----+--------+-----------+-------+
|"58;""management""|""married""| ""tertiary""| ""no""|   2143|""yes""|""no""|""unknown""|  5|""may""|     261|       1|   -1|       0|""unknown""|""no"""|
|"44;""technician""| ""single""|""secondary""| ""no""|     29|""yes""|""no""|""unknown""|  5|""may""|     151|       1|   -1|       0|""unknown""|""no"""|
+------------------+-----------+-------------+-------+-------+-------+------+-----------+---+-------+--------+--------+-----+--------+-----------+-------+
only showing top 2 rows



In [23]:
x=df.select(df['age_job'])

In [24]:
x.show()

+--------------------+
|             age_job|
+--------------------+
|  "58;""management""|
|  "44;""technician""|
|"33;""entrepreneur""|
| "47;""blue-collar""|
|     "33;""unknown""|
|  "35;""management""|
|  "28;""management""|
|"42;""entrepreneur""|
|     "58;""retired""|
|  "43;""technician""|
|      "41;""admin.""|
|      "29;""admin.""|
|  "53;""technician""|
|  "58;""technician""|
|    "57;""services""|
|     "51;""retired""|
|      "45;""admin.""|
| "57;""blue-collar""|
|     "60;""retired""|
|    "33;""services""|
+--------------------+
only showing top 20 rows



In [25]:
df.show(2)

+------------------+-----------+-------------+-------+-------+-------+------+-----------+---+-------+--------+--------+-----+--------+-----------+-------+
|           age_job|    marital|    education|default|balance|housing|  loan|    contact|day|  month|duration|campaign|pdays|previous|   poutcome|      y|
+------------------+-----------+-------------+-------+-------+-------+------+-----------+---+-------+--------+--------+-----+--------+-----------+-------+
|"58;""management""|""married""| ""tertiary""| ""no""|   2143|""yes""|""no""|""unknown""|  5|""may""|     261|       1|   -1|       0|""unknown""|""no"""|
|"44;""technician""| ""single""|""secondary""| ""no""|     29|""yes""|""no""|""unknown""|  5|""may""|     151|       1|   -1|       0|""unknown""|""no"""|
+------------------+-----------+-------------+-------+-------+-------+------+-----------+---+-------+--------+--------+-----+--------+-----------+-------+
only showing top 2 rows



In [26]:

y=x.withColumn("age", split(col("age_job"), ";").getItem(0)).withColumn("job", split(col("age_job"), ";").getItem(1))


In [27]:
y.show()

+--------------------+---+----------------+
|             age_job|age|             job|
+--------------------+---+----------------+
|  "58;""management""|"58|  ""management""|
|  "44;""technician""|"44|  ""technician""|
|"33;""entrepreneur""|"33|""entrepreneur""|
| "47;""blue-collar""|"47| ""blue-collar""|
|     "33;""unknown""|"33|     ""unknown""|
|  "35;""management""|"35|  ""management""|
|  "28;""management""|"28|  ""management""|
|"42;""entrepreneur""|"42|""entrepreneur""|
|     "58;""retired""|"58|     ""retired""|
|  "43;""technician""|"43|  ""technician""|
|      "41;""admin.""|"41|      ""admin.""|
|      "29;""admin.""|"29|      ""admin.""|
|  "53;""technician""|"53|  ""technician""|
|  "58;""technician""|"58|  ""technician""|
|    "57;""services""|"57|    ""services""|
|     "51;""retired""|"51|     ""retired""|
|      "45;""admin.""|"45|      ""admin.""|
| "57;""blue-collar""|"57| ""blue-collar""|
|     "60;""retired""|"60|     ""retired""|
|    "33;""services""|"33|    ""

In [28]:
df_cleaned=df

In [29]:

df_final= df_cleaned\
.withColumn("age", split(col("age_job"), ";").getItem(0))\
.withColumn('age', regexp_replace('age', '"', ''))\
.withColumn("job", split(col("age_job"), ";").getItem(1))\
.withColumn('job', regexp_replace('job', '""', ''))\
.withColumn('marital', regexp_replace('marital', '""', ''))\
.withColumn('education', regexp_replace('education', '""', ''))\
.withColumn('default', regexp_replace('default', '""', ''))\
.withColumn('balance', regexp_replace('balance', '""', ''))\
.withColumn('housing', regexp_replace('housing', '""', ''))\
.withColumn('loan', regexp_replace('loan', '""', ''))\
.withColumn('contact', regexp_replace('contact', '""', ''))\
.withColumn('day', regexp_replace('day', '""', ''))\
.withColumn('month', regexp_replace('month', '""', ''))\
.withColumn('duration', regexp_replace('duration', '""', ''))\
.withColumn('campaign', regexp_replace('campaign', '""', ''))\
.withColumn('pdays', regexp_replace('pdays', '""', ''))\
.withColumn('previous', regexp_replace('previous', '""', ''))\
.withColumn('poutcome', regexp_replace('poutcome', '""', ''))\
.withColumn('y', regexp_replace('y', '"', ''))

In [30]:
columns_to_drop = ['age_job']
df_final1 = df_final.drop(*columns_to_drop)

#df_final.drop('age_job')

In [31]:
df_final1.show(2)

+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+---+----------+
|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|age|       job|
+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+---+----------+
|married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no| 58|management|
| single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no| 44|technician|
+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+---+----------+
only showing top 2 rows



In [32]:
df_final1.printSchema()

root
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)
 |-- age: string (nullable = true)
 |-- job: string (nullable = true)



In [33]:
# df_final1.withColumn("success_rate", walmart["y"]/walmart["Volume"]).show()

In [34]:
df_final1.show(2)

+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+---+----------+
|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|age|       job|
+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+---+----------+
|married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no| 58|management|
| single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no| 44|technician|
+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+---+----------+
only showing top 2 rows



In [40]:
df_final1.groupBy("y").count().show()

+---+-----+
|  y|count|
+---+-----+
| no|39922|
|yes| 5289|
+---+-----+



In [43]:
q1= df_final1.describe()

In [45]:
q1.show()

+-------+--------+---------+-------+------------------+-------+-----+--------+-----------------+-----+------------------+-----------------+------------------+------------------+--------+-----+-----------------+-------+
|summary| marital|education|default|           balance|housing| loan| contact|              day|month|          duration|         campaign|             pdays|          previous|poutcome|    y|              age|    job|
+-------+--------+---------+-------+------------------+-------+-----+--------+-----------------+-----+------------------+-----------------+------------------+------------------+--------+-----+-----------------+-------+
|  count|   45211|    45211|  45211|             45211|  45211|45211|   45211|            45211|45211|             45211|            45211|             45211|             45211|   45211|45211|            45211|  45211|
|   mean|    null|     null|   null|1362.2720576850766|   null| null|    null|15.80641879188693| null| 258.1630797814691|2.7

In [ ]:
#2. Give marketing success rate (No. of people subscribed / total no. of entries) Give marketing failure rate


In [46]:
df=df_final1

In [47]:
df.createOrReplaceTempView("df")

In [48]:
df.createOrReplaceTempView('df')


In [ ]:
#2.a) Marketing success Rate


spark.sql("(select count(*) as not_subscribed from df where y='yes') /(select count(*) as total from df)").show() 

In [ ]:
#2.a) Marketing Failure Rate


spark.sql("(select count(*) as not_subscribed from df where y='no') /(select count(*) as total from df)").show() 

In [51]:
df_final1.show(1)

+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+---+----------+
|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|age|       job|
+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+---+----------+
|married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no| 58|management|
+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+---+----------+
only showing top 1 row



In [ ]:
#1. Give the maximum, mean, and minimum age of the average targeted customer


In [ ]:
df_final1.agg({"age": "max"}).collect()[0][0]



In [ ]:
df_final1.agg({"age": "min"}).collect()[0][0]

In [ ]:
df_final1.agg({"age": "avg"}).collect()[0][0]

In [ ]:
# 2. Check the quality of customers by checking average balance, median balance of customers

In [28]:
spark.sql("SELECT max(balance) FROM df").show() 

+------------+
|max(balance)|
+------------+
|        9997|
+------------+



In [30]:
spark.sql("SELECT percentile_approx(balance, 0.5) FROM df").show() 

+----------------------------------------------------------------------+
|percentile_approx(CAST(balance AS DOUBLE), CAST(0.5 AS DOUBLE), 10000)|
+----------------------------------------------------------------------+
|                                                                 448.0|
+----------------------------------------------------------------------+



In [41]:
import matplotlib.pyplot as plt
import pandas as pd



In [42]:
# 3. Check if age matters in marketing subscription for deposit

In [58]:
q3 = spark.sql("select age, count(*) as number from df where y='yes' group by age order by number desc ")


q3.show()


+---+------+
|age|number|
+---+------+
| 32|   221|
| 30|   217|
| 33|   210|
| 35|   209|
| 31|   206|
| 34|   198|
| 36|   195|
| 29|   171|
| 37|   170|
| 28|   162|
| 38|   144|
| 39|   143|
| 27|   141|
| 26|   134|
| 41|   120|
| 46|   118|
| 40|   116|
| 25|   113|
| 47|   113|
| 42|   111|
+---+------+
only showing top 20 rows



In [ ]:
#4. Check if marital status mattered for a subscription to deposit

In [63]:
spark.sql("select marital, count(*) as number from df where y='yes' group by marital order by number desc ").show() 

+--------+------+
| marital|number|
+--------+------+
| married|  2755|
|  single|  1912|
|divorced|   622|
+--------+------+



In [ ]:
#5. Check if age and marital status together mattered for a subscription to deposit scheme

In [64]:
spark.sql("select age, marital, count(*) as number from df where y='yes' group by age,marital order by number desc ").show() 


+---+-------+------+
|age|marital|number|
+---+-------+------+
| 30| single|   151|
| 28| single|   138|
| 29| single|   133|
| 32| single|   124|
| 26| single|   121|
| 34|married|   118|
| 31| single|   111|
| 27| single|   110|
| 35|married|   101|
| 36|married|   100|
| 25| single|    99|
| 37|married|    98|
| 33| single|    97|
| 33|married|    97|
| 32|married|    87|
| 39|married|    87|
| 38|married|    86|
| 35| single|    84|
| 47|married|    83|
| 46|married|    80|
+---+-------+------+
only showing top 20 rows



In [ ]:
#6. Do feature engineering for the bank and find the right age effect on the campaign